# 导入库

In [1]:
import tensorflow as tf
import keras 
import cv2
import os
import scipy
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import os
import h5py
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

Using TensorFlow backend.


# 图片命名

In [ ]:
#coding:utf8
import os;

def rename():
         i=0
         path="picture//牛鲨";
         filelist=os.listdir(path)#该文件夹下所有的文件（包括文件夹）
         for files in filelist:#遍历所有文件
             i=i+1
             Olddir=os.path.join(path,files);#原来的文件路径                
             if os.path.isdir(Olddir):#如果是文件夹则跳过
                     continue;
             filename=os.path.splitext(files)[0];#文件名
             filetype=os.path.splitext(files)[1];#文件扩展名
             fn = "bull_"
             Newdir=os.path.join(path,fn+str(i)+filetype);#新的文件路径
             os.rename(Olddir,Newdir)#重命名
rename()

# 数据处理

In [2]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.15,
    height_shift_range=0.15,
    zoom_range=0.15,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
dirs = os.listdir("allshark")
print(len(dirs))
for filename in dirs:
    img = load_img("allshark//{}".format(filename))
    x = img_to_array(img)
    # print(x.shape)
    x = x.reshape((1,) + x.shape) #datagen.flow要求rank为4
    # print(x.shape)
    datagen.fit(x)
    prefix = filename.split('.')[0]
    print(prefix)
    counter = 0
    for batch in datagen.flow(x, batch_size=4 , save_to_dir='generater_pic', save_prefix=prefix, save_format='jpg'):
        counter += 1
        if counter > 100:
            break  # 否则生成器会退出循环

262
billhead_22
billhead_36
mako_24
jaws_32
mako_30
jaws_26
mako_18
jaws_4
billhead_8
whale_40
whale_55
whale_41
jaws_5
bull_4
billhead_9
mako_19
bull_5
jaws_33
whale_54
billhead_35
jaws_27
billhead_21
mako_25
mako_31
billhead_37
jaws_25
mako_27
mako_33
jaws_19
jaws_31
whale_57
jaws_7
whale_42
whale_56
bull_6
whale_43
bull_7
jaws_6
jaws_30
jaws_24
billhead_30
mako_26
mako_32
jaws_18
billhead_34
jaws_34
billhead_18
billhead_20
jaws_20
billhead_24
bull_48
mako_22
whale_53
whale_47
bull_3
jaws_2
mako_36
whale_52
whale_46
bull_49
bull_2
jaws_3
jaws_35
mako_37
mako_23
billhead_25
jaws_21
billhead_31
billhead_19
jaws_37
billhead_27
billhead_33
mako_35
whale_44
jaws_23
jaws_1
whale_50
mako_21
whale_51
mako_20
jaws_22
whale_45
bull_1
mako_34
jaws_36
billhead_26
billhead_32
mako_53
bull_39
billhead_41
mako_1
whale_22
whale_37
whale_36
mako_47
bull_11
whale_23
bull_10
billhead_42
mako_46
billhead_40
mako_2
mako_50
mako_44
bull_12
bull_38
mako_52
whale_21
whale_35
whale_34
billhead_47
whale_20
ma

In [2]:
def resize_img():
    dirs = os.listdir("generater_pic")
    for filename in dirs:
        im = tf.gfile.FastGFile("generater_pic//{}".format(filename), 'rb').read()
        # print("正在处理第%d张照片"%counter)
        with tf.Session() as sess:
            img_data = tf.image.decode_jpeg(im)
            image_float = tf.image.convert_image_dtype(img_data, tf.float32)
            resized = tf.image.resize_images(image_float, [64, 64], method=3)
            resized_im = resized.eval()
            # new_mat = np.asarray(resized_im).reshape(1, 64, 64, 3)
            scipy.misc.imsave("resized_img//{}".format(filename),resized_im)

In [ ]:
resize_img() 

/private/sxu/.conda/envs/Tensorflow-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  if sys.path[0] == '':


In [2]:
#图片转h5文件
    #test
    # data = h5py.File("dataset//data.h5","r")
    # X_data = data['X']
    # print(X_data.shape)
    # Y_data = data['Y']
    # print(Y_data[123])
    # image = Image.fromarray(X_data[123]) #矩阵转图片并显示
    # image.show()

In [3]:
dirs = os.listdir("resized_img")
Y = [] #label
X = [] #data
i = 0
print(len(dirs))
for filename in dirs:
    print(i/len(dirs))
    i = i+1
    label = str(filename.split('_')[0]).encode()
    #print(filename)
    #print(label)
    Y.append(label)
    im = Image.open("resized_img//{}".format(filename)).convert('RGB')
    mat = np.asarray(im) #image 转矩阵
    X.append(mat)


9630
0.0
0.00010384215991692627
0.00020768431983385254
0.00031152647975077883
0.0004153686396677051
0.0005192107995846313
0.0006230529595015577
0.0007268951194184839
0.0008307372793354102
0.0009345794392523365
0.0010384215991692627
0.001142263759086189
0.0012461059190031153
0.0013499480789200416
0.0014537902388369677
0.001557632398753894
0.0016614745586708203
0.0017653167185877467
0.001869158878504673
0.001973001038421599
0.0020768431983385254
0.0021806853582554517
0.002284527518172378
0.0023883696780893043
0.0024922118380062306
0.002596053997923157
0.0026998961578400833
0.002803738317757009
0.0029075804776739354
0.0030114226375908618
0.003115264797507788
0.0032191069574247144
0.0033229491173416407
0.003426791277258567
0.0035306334371754933
0.0036344755970924196
0.003738317757009346
0.0038421599169262723
0.003946002076843198
0.004049844236760125
0.004153686396677051
0.0042575285565939775
0.004361370716510903
0.004465212876427829
0.004569055036344756
0.004672897196261682
0.0047767393561

0.0490134994807892
0.04911734164070613
0.049221183800623056
0.04932502596053998
0.0494288681204569
0.04953271028037383
0.049636552440290756
0.04974039460020768
0.04984423676012461
0.049948078920041536
0.05005192107995846
0.05015576323987539
0.050259605399792316
0.05036344755970924
0.05046728971962617
0.050571131879543096
0.05067497403946002
0.05077881619937695
0.05088265835929388
0.050986500519210796
0.05109034267912772
0.05119418483904465
0.05129802699896158
0.0514018691588785
0.05150571131879543
0.05160955347871236
0.051713395638629284
0.05181723779854621
0.05192107995846314
0.052024922118380064
0.05212876427829699
0.05223260643821392
0.052336448598130844
0.052440290758047764
0.05254413291796469
0.05264797507788162
0.052751817237798544
0.05285565939771547
0.0529595015576324
0.053063343717549324
0.05316718587746625
0.05327102803738318
0.053374870197300105
0.05347871235721703
0.05358255451713396
0.053686396677050885
0.05379023883696781
0.05389408099688474
0.05399792315680166
0.05410176

0.10051921079958463
0.10062305295950155
0.10072689511941849
0.1008307372793354
0.10093457943925234
0.10103842159916926
0.10114226375908619
0.10124610591900311
0.10134994807892005
0.10145379023883697
0.1015576323987539
0.10166147455867082
0.10176531671858775
0.10186915887850467
0.10197300103842159
0.10207684319833853
0.10218068535825545
0.10228452751817238
0.1023883696780893
0.10249221183800623
0.10259605399792315
0.10269989615784009
0.102803738317757
0.10290758047767394
0.10301142263759086
0.1031152647975078
0.10321910695742471
0.10332294911734165
0.10342679127725857
0.10353063343717549
0.10363447559709242
0.10373831775700934
0.10384215991692627
0.1039460020768432
0.10404984423676013
0.10415368639667705
0.10425752855659398
0.1043613707165109
0.10446521287642783
0.10456905503634475
0.10467289719626169
0.10477673935617861
0.10488058151609553
0.10498442367601246
0.10508826583592938
0.10519210799584632
0.10529595015576323
0.10539979231568017
0.10550363447559709
0.10560747663551402
0.105711

0.14953271028037382
0.14963655244029075
0.1497403946002077
0.14984423676012462
0.14994807892004153
0.15005192107995846
0.1501557632398754
0.15025960539979233
0.15036344755970923
0.15046728971962617
0.1505711318795431
0.15067497403946
0.15077881619937694
0.15088265835929388
0.1509865005192108
0.15109034267912771
0.15119418483904465
0.15129802699896158
0.15140186915887852
0.15150571131879542
0.15160955347871236
0.1517133956386293
0.15181723779854622
0.15192107995846313
0.15202492211838006
0.152128764278297
0.1522326064382139
0.15233644859813084
0.15244029075804777
0.1525441329179647
0.1526479750778816
0.15275181723779854
0.15285565939771548
0.1529595015576324
0.15306334371754932
0.15316718587746625
0.15327102803738318
0.15337487019730012
0.15347871235721702
0.15358255451713396
0.1536863966770509
0.1537902388369678
0.15389408099688473
0.15399792315680166
0.1541017653167186
0.1542056074766355
0.15430944963655244
0.15441329179646937
0.1545171339563863
0.1546209761163032
0.15472481827622014


0.1995846313603323
0.1996884735202492
0.19979231568016614
0.19989615784008308
0.2
0.20010384215991692
0.20020768431983385
0.20031152647975078
0.20041536863966772
0.20051921079958462
0.20062305295950156
0.2007268951194185
0.2008307372793354
0.20093457943925233
0.20103842159916926
0.2011422637590862
0.2012461059190031
0.20134994807892004
0.20145379023883697
0.2015576323987539
0.2016614745586708
0.20176531671858774
0.20186915887850468
0.2019730010384216
0.20207684319833852
0.20218068535825545
0.20228452751817239
0.2023883696780893
0.20249221183800623
0.20259605399792316
0.2026998961578401
0.202803738317757
0.20290758047767393
0.20301142263759087
0.2031152647975078
0.2032191069574247
0.20332294911734164
0.20342679127725857
0.2035306334371755
0.2036344755970924
0.20373831775700935
0.20384215991692628
0.20394600207684319
0.20404984423676012
0.20415368639667705
0.204257528556594
0.2043613707165109
0.20446521287642783
0.20456905503634476
0.2046728971962617
0.2047767393561786
0.2048805815160955

...

0.7892004153686397
0.7893042575285566
0.7894080996884735
0.7895119418483905
0.7896157840083073
0.7897196261682243
0.7898234683281412
0.7899273104880582
0.7900311526479751
0.790134994807892
0.7902388369678089
0.7903426791277258
0.7904465212876428
0.7905503634475597
0.7906542056074767
0.7907580477673936
0.7908618899273104
0.7909657320872274
0.7910695742471443
0.7911734164070613
0.7912772585669782
0.7913811007268952
0.791484942886812
0.791588785046729
0.7916926272066459
0.7917964693665628
0.7919003115264798
0.7920041536863967
0.7921079958463136
0.7922118380062305
0.7923156801661475
0.7924195223260644
0.7925233644859813
0.7926272066458983
0.7927310488058151
0.7928348909657321
0.792938733125649
0.793042575285566
0.7931464174454829
0.7932502596053997
0.7933541017653167
0.7934579439252336
0.7935617860851506
0.7936656282450675
0.7937694704049845
0.7938733125649013
0.7939771547248182
0.7940809968847352
0.7941848390446521
0.7942886812045691
0.794392523364486
0.794496365524403
0.7946002076843198


0.8358255451713396
0.8359293873312564
0.8360332294911734
0.8361370716510903
0.8362409138110073
0.8363447559709242
0.8364485981308412
0.836552440290758
0.836656282450675
0.8367601246105919
0.8368639667705088
0.8369678089304258
0.8370716510903426
0.8371754932502596
0.8372793354101765
0.8373831775700935
0.8374870197300104
0.8375908618899273
0.8376947040498443
0.8377985462097611
0.8379023883696781
0.838006230529595
0.838110072689512
0.8382139148494289
0.8383177570093457
0.8384215991692627
0.8385254413291796
0.8386292834890966
0.8387331256490135
0.8388369678089305
0.8389408099688473
0.8390446521287642
0.8391484942886812
0.8392523364485981
0.8393561786085151
0.839460020768432
0.839563862928349
0.8396677050882658
0.8397715472481828
0.8398753894080997
0.8399792315680166
0.8400830737279336
0.8401869158878504
0.8402907580477674
0.8403946002076843
0.8404984423676013
0.8406022845275182
0.840706126687435
0.840809968847352
0.8409138110072689
0.8410176531671859
0.8411214953271028
0.8412253374870198
0

0.8836967808930426
0.8838006230529595
0.8839044652128765
0.8840083073727933
0.8841121495327103
0.8842159916926272
0.8843198338525441
0.8844236760124611
0.884527518172378
0.884631360332295
0.8847352024922118
0.8848390446521288
0.8849428868120457
0.8850467289719626
0.8851505711318796
0.8852544132917964
0.8853582554517134
0.8854620976116303
0.8855659397715473
0.8856697819314642
0.885773624091381
0.885877466251298
0.8859813084112149
0.8860851505711319
0.8861889927310488
0.8862928348909658
0.8863966770508827
0.8865005192107995
0.8866043613707165
0.8867082035306334
0.8868120456905504
0.8869158878504673
0.8870197300103843
0.8871235721703011
0.8872274143302181
0.887331256490135
0.8874350986500519
0.8875389408099689
0.8876427829698857
0.8877466251298027
0.8878504672897196
0.8879543094496366
0.8880581516095535
0.8881619937694704
0.8882658359293873
0.8883696780893042
0.8884735202492212
0.8885773624091381
0.8886812045690551
0.888785046728972
0.8888888888888888
0.8889927310488058
0.8890965732087227

0.930841121495327
0.930944963655244
0.9310488058151609
0.9311526479750779
0.9312564901349948
0.9313603322949118
0.9314641744548287
0.9315680166147456
0.9316718587746625
0.9317757009345794
0.9318795430944964
0.9319833852544133
0.9320872274143303
0.9321910695742471
0.9322949117341641
0.932398753894081
0.9325025960539979
0.9326064382139149
0.9327102803738317
0.9328141225337487
0.9329179646936656
0.9330218068535826
0.9331256490134995
0.9332294911734164
0.9333333333333333
0.9334371754932502
0.9335410176531672
0.9336448598130841
0.9337487019730011
0.933852544132918
0.9339563862928348
0.9340602284527518
0.9341640706126687
0.9342679127725857
0.9343717549325026
0.9344755970924196
0.9345794392523364
0.9346832814122533
0.9347871235721703
0.9348909657320872
0.9349948078920042
0.935098650051921
0.935202492211838
0.9353063343717549
0.9354101765316719
0.9355140186915888
0.9356178608515057
0.9357217030114227
0.9358255451713395
0.9359293873312565
0.9360332294911734
0.9361370716510904
0.9362409138110073

0.9773624091381101
0.977466251298027
0.9775700934579439
0.9776739356178609
0.9777777777777777
0.9778816199376947
0.9779854620976116
0.9780893042575286
0.9781931464174455
0.9782969885773624
0.9784008307372793
0.9785046728971962
0.9786085150571132
0.9787123572170301
0.9788161993769471
0.978920041536864
0.9790238836967808
0.9791277258566978
0.9792315680166147
0.9793354101765317
0.9794392523364486
0.9795430944963656
0.9796469366562824
0.9797507788161994
0.9798546209761163
0.9799584631360332
0.9800623052959502
0.980166147455867
0.980269989615784
0.9803738317757009
0.9804776739356179
0.9805815160955348
0.9806853582554517
0.9807892004153687
0.9808930425752855
0.9809968847352025
0.9811007268951194
0.9812045690550364
0.9813084112149533
0.9814122533748701
0.9815160955347871
0.981619937694704
0.981723779854621
0.9818276220145379
0.9819314641744549
0.9820353063343717
0.9821391484942886
0.9822429906542056
0.9823468328141225
0.9824506749740395
0.9825545171339564
0.9826583592938734
0.9827622014537902

In [3]:
file = h5py.File("dataset//data.h5","w")
file.create_dataset('X', data=np.array(X))
file.create_dataset('Y', data=np.array(Y))
file.close()

NameError: name 'X' is not defined

In [4]:
data = h5py.File("dataset//data.h5","r")
X_data = data['X']
print(X_data.shape)
Y_data = data['Y']
print(Y_data[1])
image = Image.fromarray(X_data[1]) #矩阵转图片并显示
image.show()

KeyError: "Unable to open object (object 'X' doesn't exist)"

In [5]:
#def load_dataset():
#划分训练集、测试集
data = h5py.File("dataset//data.h5","r")
X_data = np.array(data['X']) #data['X']是h5py._hl.dataset.Dataset类型，转化为array
Y_data = np.array(data['Y'])
# print(type(X_data))
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, train_size=0.9, test_size=0.1, random_state=22)

KeyError: "Unable to open object (object 'X' doesn't exist)"

In [60]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(8667, 64, 64, 3)
(8667, 2)
(963, 64, 64, 3)
(963, 2)


In [2]:
def image_to_h5():
    dirs = os.listdir("resized_img")
    Y = [] #label
    X = [] #data
    i = 0
    print(len(dirs))
    for filename in dirs:
        print(i/len(dirs))
        i = i+1
        label = str(filename.split('_')[0]).encode()
        Y.append(label)
        im = Image.open("resized_img//{}".format(filename)).convert('RGB')
        mat = np.asarray(im) #image 转矩阵
        X.append(mat)

    file = h5py.File("dataset//data.h5","w")
    file.create_dataset('X', data=np.array(X))
    file.create_dataset('Y', data=np.array(Y))
    file.close()

In [3]:
image_to_h5()

9630
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [4]:
def load_dataset():
    data = h5py.File("dataset//data.h5","r")
    X_data = np.array(data['X']) #data['X']是h5py._hl.dataset.Dataset类型，转化为array
    Y_data = np.array(data['Y'])
    # print(type(X_data))
    X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, train_size=0.9, test_size=0.1, random_state=22)
    # print(X_train.shape)
    # print(y_train[456])
    # image = Image.fromarray(X_train[456])
    # image.show()
    # y_train = y_train.reshape(1,y_train.shape[0])
    # y_test = y_test.reshape(1,y_test.shape[0])
    # print(X_train[0])
    X_train = X_train / 255.  # 归一化
    X_test = X_test / 255.
    # print(X_train[0])
    # one-hot
    encoder = LabelEncoder()
    train_Y = encoder.fit_transform(y_train)
    test_Y = encoder.fit_transform(y_test)
    y_train = np_utils.to_categorical(train_Y)
    y_test = np_utils.to_categorical(test_Y)
    print(X_train.shape)
    print(y_train.shape)
    print(X_test.shape)
    print(y_test.shape)

    return X_train, X_test, y_train, y_test

In [5]:
X_train, X_test, y_train, y_test = load_dataset()

(8667, 64, 64, 3)
(8667, 2)
(963, 64, 64, 3)
(963, 2)


In [8]:
print(type(y_train))
y_train[0:15]

<class 'numpy.ndarray'>


array([[ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.]], dtype=float32)

# Public function

In [9]:
def weight_variable(shape):
    tf.set_random_seed(1)
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

In [10]:
def bias_variable(shape):
    return tf.Variable(tf.constant(0.0, shape=shape))

In [11]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

In [12]:
def max_pool_2x2(z):
    return tf.nn.max_pool(z, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [13]:
def random_mini_batches(X, Y, mini_batch_size=16, seed=0):
    """
    Creates a list of random minibatches from (X, Y)

    Arguments:
    X -- input data, of shape (input size, number of examples)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples)
    mini_batch_size - size of the mini-batches, integer
    seed -- this is only for the purpose of grading, so that you're "random minibatches are the same as ours.

    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """

    m = X.shape[0]  # number of training examples
    mini_batches = []
    np.random.seed(seed)

    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation]
    shuffled_Y = Y[permutation,:].reshape((m, Y.shape[1]))

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m / mini_batch_size)  # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[k * mini_batch_size: k * mini_batch_size + mini_batch_size]
        mini_batch_Y = shuffled_Y[k * mini_batch_size: k * mini_batch_size + mini_batch_size]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)

    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[num_complete_minibatches * mini_batch_size: m]
        mini_batch_Y = shuffled_Y[num_complete_minibatches * mini_batch_size: m]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)

    return mini_batches


# Input Layer

In [14]:
with tf.name_scope('Input_Layer'):
    X = tf.placeholder(tf.float32, [None, 64, 64, 3], name="input_x")
    y = tf.placeholder(tf.float32, [None, 2], name="input_y")
    kp = tf.placeholder_with_default(1.0, shape=(), name="keep_prob")
    lam = tf.placeholder(tf.float32, name="lamda")

# Convolutional Layer 1

In [15]:
with tf.name_scope('C1_Conv'):
    W_conv1 = weight_variable([5,5,3,32])
    b_conv1 = bias_variable([32])
    z1 = tf.nn.relu(conv2d(X, W_conv1) + b_conv1)

In [16]:
with tf.name_scope('C1_Pool'):
    maxpool1 = max_pool_2x2(z1) #max_pool1完后maxpool1维度为[?,32,32,32]

# Convolutional Layer 2

In [17]:
with tf.name_scope('C2_Conv'):
    W_conv2 = weight_variable([5,5,32,64])
    b_conv2 = bias_variable([64])
    z2 = tf.nn.relu(conv2d(maxpool1, W_conv2) + b_conv2)

In [18]:
with tf.name_scope('C2_Pool'):
    maxpool2 = max_pool_2x2(z2) #max_pool2,shape [?,16,16,64]

# Convolutional Layer 3

In [19]:
with tf.name_scope('C3_Conv'):
    W_conv3 = weight_variable([5, 5, 64, 128])
    b_conv3 = bias_variable([128])
    z3 = tf.nn.relu(conv2d(maxpool2, W_conv3) + b_conv3)

In [20]:
with tf.name_scope('C3_Pool'):
    maxpool3 = max_pool_2x2(z3)  # max_pool3,shape [?,8,8,128]

# Fully Connected Layer

In [21]:
with tf.name_scope('D_Flat'):
    maxpool3_flat = tf.reshape(maxpool3, [-1, 8*8*128])

In [22]:
with tf.name_scope('D_Hidden_Layer'):    
    W_fc1 = weight_variable([8*8*128, 200])
    b_fc1 = bias_variable([200])
    z_fc1 = tf.nn.relu(tf.matmul(maxpool3_flat, W_fc1) + b_fc1)
    z_fc1_drop = tf.nn.dropout(z_fc1, keep_prob=kp)            #Drop_out kp=0.8

# Output  Layer

In [29]:
with tf.name_scope('Output_Layer'):
    W_fc2 = weight_variable([200, 2])
    b_fc2 = bias_variable([2])
    z_fc2 = tf.add(tf.matmul(z_fc1_drop, W_fc2),b_fc2, name="outlayer")
    prob = tf.nn.softmax(z_fc2, name="probability")

# Optimazer

In [30]:
with tf.name_scope("optimizer"):
    regularizer = tf.contrib.layers.l2_regularizer(lam)       # L2正则 lam=0.5
    regularization = regularizer(W_fc1) + regularizer(W_fc2)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=z_fc2)) + regularization
    train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

# Evaluate Model

In [25]:
with tf.name_scope("evaluate_model"):
    pred = tf.argmax(prob, 1, output_type="int32", name="predict")  # 输出结点名称predict方便后面保存为pb文件
    correct_prediction = tf.equal(pred, tf.argmax(y, 1, output_type='int32'))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [26]:
epoch_list=[];loss_list=[];
from time import time
startTime=time()

In [31]:
import math
def cnn_model(X_train, y_train, X_test, y_test, keep_prob, lamda=0.5, num_epochs = 450, minibatch_size = 20):
    init = tf.global_variables_initializer()
    tf.set_random_seed(1)  # to keep consistent results
    seed = 0
    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(num_epochs):
            seed = seed + 1
            epoch_cost = 0.
            num_minibatches = int(X_train.shape[0] / minibatch_size)
            minibatches = random_mini_batches(X_train, y_train, minibatch_size, seed)
            start_epo = time()
            for minibatch in minibatches:
                (minibatch_X, minibatch_Y) = minibatch
                _, minibatch_cost = sess.run([train, cost], feed_dict={X: minibatch_X, y: minibatch_Y, kp: keep_prob, lam: lamda})

                epoch_cost += minibatch_cost / num_minibatches

            epoch_list.append(epoch)
            loss_list.append(epoch_cost)
            end_epo = time() - start_epo
            if epoch % 10 == 0:
                print("Cost after epoch %i: %f spend %d s" % (epoch, epoch_cost, end_epo))

        train_acc = accuracy.eval(feed_dict={X: X_train, y: y_train, kp: 0.8, lam: lamda})
        test_acc = accuracy.eval(feed_dict={X: X_test, y: y_test, lam: lamda})
        print("train accuracy", train_acc)
        print("test accuracy", test_acc)
        
        duration =time()-startTime
        print("Train Finished takes:",duration) 

tf.reset_default_graph()

In [32]:
cnn_model(X_train, y_train, X_test, y_test,keep_prob=0.8)

Cost after epoch 0: 372.312835 
Cost after epoch 10: 0.148621 
Cost after epoch 20: 0.083549 
Cost after epoch 30: 0.122189 
Cost after epoch 40: 0.061700 
Cost after epoch 50: 0.162271 
Cost after epoch 60: 0.112326 
Cost after epoch 70: 0.074159 
Cost after epoch 80: 0.049136 
Cost after epoch 90: 0.086770 
Cost after epoch 100: 0.060377 
Cost after epoch 110: 0.021280 
Cost after epoch 120: 0.131438 
Cost after epoch 130: 0.165240 
Cost after epoch 140: 0.018229 
Cost after epoch 150: 0.112668 
Cost after epoch 160: 0.214789 
Cost after epoch 170: 0.098081 
Cost after epoch 180: 0.078775 
Cost after epoch 190: 0.018031 
Cost after epoch 200: 0.017662 
Cost after epoch 210: 0.154246 
Cost after epoch 220: 0.017465 
Cost after epoch 230: 0.071084 
Cost after epoch 240: 0.041998 
Cost after epoch 250: 0.245037 
Cost after epoch 260: 0.321798 
Cost after epoch 270: 0.012690 
Cost after epoch 280: 0.023441 
Cost after epoch 290: 0.014912 
Cost after epoch 300: 0.032601 
Cost after epoch 

ResourceExhaustedError: OOM when allocating tensor with shape[8667,32,64,64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: C1_Conv/Conv2D = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](C1_Conv/Conv2D-0-TransposeNHWCToNCHW-LayoutOptimizer, C1_Conv/Variable/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: evaluate_model/Mean/_29 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_92_evaluate_model/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
fig = plt.gcf()
fig.set_size_inches(4,2)
plt.plot(epoch_list, loss_list, label = 'loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss'], loc='upper left')

#save model
saver = tf.train.Saver({'W_conv1':W_conv1, 'b_conv1':b_conv1, 'W_conv2':W_conv2, 'b_conv2':b_conv2,
                        'W_fc1':W_fc1, 'b_fc1':b_fc1, 'W_fc2':W_fc2, 'b_fc2':b_fc2})
saver.save(sess, "model_500_200_c3//cnn_model.ckpt")

#将训练好的模型保存为.pb文件，方便在Android studio中使用
output_graph_def = graph_util.convert_variables_to_constants(sess, sess.graph_def, output_node_names=['predict'])

with tf.gfile.FastGFile('model_500_200_c3//digital_gesture.pb', mode='wb') as f:  # ’wb’中w代表写文件，b代表将数据以二进制方式写入文件。
    f.write(output_graph_def.SerializeToString())